<a href="https://colab.research.google.com/github/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/viewers_deployment/OHIF_FireBase_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prerequisites

This notebook complements the tutorial available in this document:

https://docs.google.com/document/d/1v4Syu_yOV6yH--QBLGzsL9fJ7-XyD1CnQu4iTIoPVD8/edit#

You must complete the prerequisites discussed in that document before proceeding with this notebook!

Please use the document above for providing feedback or asking questions!



## Install and configure Firebase command line tools

In this step we install the command line tools provided by Firebase to populate the webapp hosting project.


In [ ]:
!curl -sL https://firebase.tools | bash

-- Checking for existing firebase-tools on PATH...
-- Checking your machine type...
-- Downloading binary from https://firebase.tools/bin/linux/latest
######################################################################## 100.0%
-- Setting permissions on binary...
-- Checking your PATH variable...
-- firebase-tools@9.22.0 is now installed
-- All Done!


In the next step we configure setup of Firebase on the VM. When prompted regarding error reporting information, double click next to the prompt, and type your selection. Next, follow the authorization URL to grant the necessary permissions to Firebase, then double-click next to the "Paste authorization code", paste the code and hit "Enter" to complete the process.

In [ ]:
!firebase login --no-localhost

## Clone OHIF Viewer source code

In [ ]:
!git clone https://github.com/OHIF/Viewers.git

Cloning into 'Viewers'...
remote: Enumerating objects: 144692, done.
remote: Counting objects: 100% (1079/1079), done.
remote: Compressing objects: 100% (571/571), done.
remote: Total 144692 (delta 660), reused 817 (delta 478), pack-reused 143613
Receiving objects: 100% (144692/144692), 227.74 MiB | 25.36 MiB/s, done.
Resolving deltas: 100% (102889/102889), done.


If you would like to have the DICOM tag browser button in the toolbar, uncomment the following two lines in the source code: Viewers/platform/viewer/src/index.js:

```
//import OHIFDicomTagBrowserExtension from '@ohif/extension-dicom-tag-browser';
```


```
 //OHIFDicomTagBrowserExtension,
 ```

## Initialize Firebase configuration

You should have created a Firebase project as a prerequisite step. Let's first confirm this is indeed the case.

In [ ]:
!firebase projects:list

⠋ Preparing the list of your Firebase projects⠙ Preparing the list of your Firebase projects⠹ Preparing the list of your Firebase projects⠸ Preparing the list of your Firebase projects✔ Preparing the list of your Firebase projects
┌──────────────────────┬──────────────┬────────────────┬──────────────────────┐
│ Project Display Name │ Project ID   │ Project Number │ Resource Location ID │
├──────────────────────┼──────────────┼────────────────┼──────────────────────┤
│ idc-tester-1         │ idc-tester-1 │ 201217755281   │ [Not specified]      │
└──────────────────────┴──────────────┴────────────────┴──────────────────────┘

1 project(s) total.


In the below, accept all the defaults. After you run the command, it will stop every time you need to make a selection. Double-click in the space right after the prompt asking for input and hit "Enter" to accept the default.

If you are unable to interact with the `firebase init` output to accept the default values, open the cell controls menu in the upper right, and choose "View output fullscreen".

<img src="https://www.dropbox.com/s/bvmdbeirhe0pmuc/view_fullscreen.png?raw=1" alt="new" width="400"/>

In [ ]:
!cd /content/Viewers && firebase init hosting


     ######## #### ########  ######## ########     ###     ######  ########
     ##        ##  ##     ## ##       ##     ##  ##   ##  ##       ##
     ######    ##  ########  ######   ########  #########  ######  ######
     ##        ##  ##    ##  ##       ##     ## ##     ##       ## ##
     ##       #### ##     ## ######## ########  ##     ##  ######  ########

You're about to initialize a Firebase project in this directory:

  /content/Viewers

Before we get started, keep in mind:

  * You are currently outside your home directory


=== Project Setup

First, let's associate this project directory with a Firebase project.
You can create multiple project aliases by running firebase use --add, 
but for now we'll just set up a default project.

? Please select an option: (Use arrow keys)
❯ Use an existing project 
  Create a new project 
  Add Firebase to an existing Google Cloud Platform project 
  Don't set up a default project ? Please select an option: Use an existing project
? Se

In [ ]:
!wget -O /content/Viewers/firebase.json https://gist.githubusercontent.com/fedorov/987e06eeb261c2abb21743fdbd43eb0e/raw/daa55d92164def1ed884d92c93a35cd90982cf4c/firebase.json

--2021-11-18 16:06:50--  https://gist.githubusercontent.com/fedorov/987e06eeb261c2abb21743fdbd43eb0e/raw/daa55d92164def1ed884d92c93a35cd90982cf4c/firebase.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 450 [text/plain]
Saving to: ‘/content/Viewers/firebase.json’

/content/Viewers/fi 100%[===================>]     450  --.-KB/s    in 0s      

2021-11-18 16:06:50 (37.0 MB/s) - ‘/content/Viewers/firebase.json’ saved [450/450]



## Configure OHIF Viewer

In this step we will fetch the configuration file for Firebase, and place it into the right location within the OHIF Viewer source tree.

It is important that you do NOT set the `APP_CONFIG` environment variable, despite what you might see in other deployment instructions. We confirmed setting this variable skips initialization of the OHIF Viewer config file.

**THIS IS A VERY IMPORTANT STEP: please read the instructions below**

Next we replace the placeholder of the OAuth Client ID with the actual ID you created in the setup steps. In the text below, replace `MY_OAUTH_CLIENT_ID` with the ID you created.

In [ ]:
!wget -O /content/Viewers/platform/viewer/public/config/default.js https://gist.githubusercontent.com/fedorov/737c7b76c7836b6d4de72f0fc1ce4eb5/raw/57ce940a10711e88f52234d61094766b9df35b18/google.js

!sed -i 's/CLIENT_ID_PLACEHOLDER/MY_OAUTH_CLIENT_ID/' /content/Viewers/platform/viewer/public/config/default.js

--2022-04-06 18:07:51--  https://gist.githubusercontent.com/fedorov/737c7b76c7836b6d4de72f0fc1ce4eb5/raw/2b582d5ae6160f9b71b4cf592b3a8849a4529209/google.js
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1025 (1.0K) [text/plain]
Saving to: ‘/content/Viewers/platform/viewer/public/config/default.js’

/content/Viewers/pl 100%[===================>]   1.00K  --.-KB/s    in 0s      

2022-04-06 18:07:51 (50.6 MB/s) - ‘/content/Viewers/platform/viewer/public/config/default.js’ saved [1025/1025]



... and proceed with the installation and build steps of the viewer.

In [ ]:
!npm install --global yarn


> yarn@1.22.17 preinstall /tools/node/lib/node_modules/yarn
> :; (node ./preinstall.js > /dev/null 2>&1 || true)

/tools/node/bin/yarn -> /tools/node/lib/node_modules/yarn/bin/yarn.js
/tools/node/bin/yarnpkg -> /tools/node/lib/node_modules/yarn/bin/yarn.js
+ yarn@1.22.17
added 1 package in 0.898s


**next cell will take about 7 minutes to complete**

In [ ]:
!cd Viewers && yarn install && yarn run build

yarn install v1.22.17
[1/5] Validating package.json...
[2/5] Resolving packages...
warning Resolution field "@babel/runtime@7.5.5" is incompatible with requested version "@babel/runtime@^7.8.4"
warning Resolution field "@babel/runtime@7.5.5" is incompatible with requested version "@babel/runtime@^7.9.6"
warning Resolution field "@babel/runtime@7.5.5" is incompatible with requested version "@babel/runtime@7.1.2"
warning Resolution field "@babel/runtime@7.5.5" is incompatible with requested version "@babel/runtime@^7.6.0"
warning Resolution field "@babel/runtime@7.5.5" is incompatible with requested version "@babel/runtime@7.6.0"
warning Resolution field "@babel/runtime@7.5.5" is incompatible with requested version "@babel/runtime@^7.6.3"
warning Resolution field "@babel/runtime@7.5.5" is incompatible with requested version "@babel/runtime@^7.7.2"
warning Resolution field "@babel/runtime@7.5.5" is incompatible with requested version "@babel/runtime@^7.8.7"
[3/5] Fetching packages...
[] 0

Finally, we copy the built application to the `public` folder, from which it will be deployed to Firebase.

In [ ]:
%%capture
!cd /content/Viewers && rsync -avP platform/viewer/dist/ public/
#!cd /content/Viewers && cp platform/viewer/public/config/default.js public/app-config.js

In [ ]:
!cd Viewers && firebase deploy


=== Deploying to 'idc-tester-1'...

i  deploying hosting
i  hosting[idc-tester-1]: beginning deploy...
i  hosting[idc-tester-1]: found 196 files in public
⠋  hosting: hashing files [50/196] (25%)⠙  hosting: hashing files [80/196] (40%)⠹  hosting: hashing files [122/196] (62%)⠸  hosting: hashing files [172/196] (87%)⠼  hosting: hashing files [187/196] (95%)⠴  hosting: hashing files [190/196] (96%)⠦  hosting: hashing files [191/196] (97%)⠧  hosting: hashing files [192/196] (97%)⠇  hosting: hashing files [192/196] (97%)⠏  hosting: hashing files [192/196] (97%)⠋  hosting: hashing files [193/196] (98%)⠙  hosting: hashing files [194/196] (98%)⠹  hosting: hashing files [195/196] (99%)⠸  hosting: hashing files [195/196] (99%)⠼  hosting: hashing files [195/196] (99%)⠴  hosting: hashing files [195/196] (99%)⠦  hosting: hashing files [195/196] (99%)⠧  hosting: hashing files [195/196] (99%)⠇  hosting: hashing files [195/196] (99%)⠏  hosting: hashing files [195/196] (99%)⠋  hosting: hashing files 

At this point, please return to the main tutorial, and proceed to the ["Updating configuration of the Consent screen and OAuth Client" section](https://docs.google.com/document/d/1v4Syu_yOV6yH--QBLGzsL9fJ7-XyD1CnQu4iTIoPVD8/edit#heading=h.43a3r6qbdsiy) in the tutorial.

